<a href="https://colab.research.google.com/github/mshumer/gpt-prompt-engineer/blob/main/gpt_prompt_engineer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# gpt-prompt-engineer
By Matt Shumer (https://twitter.com/mattshumer_)

Github repo: https://github.com/mshumer/gpt-prompt-engineer

Generate an optimal prompt for a given task.

To generate a prompt:
1. In the first cell, add in your OpenAI key.
2. In the last cell, fill in the description of your task, up to 15 test cases, and the number of prompts to generate.
3. Run all the cells! The AI will generate a number of candidate prompts, and test them all to find the best one!

In [ ]:
!pip install openai
!pip install prettytable
!pip install tqdm

from prettytable import PrettyTable
import time
import openai
from tqdm import tqdm
import itertools

openai.api_key = "ADD YOUR KEY HERE" # enter your OpenAI API key here

In [ ]:
# K is a constant factor that determines how much ratings change
K = 32

def generate_candidate_prompts(description, test_cases, number_of_prompts):
  outputs = openai.ChatCompletion.create(
      model='gpt-4', # change this to gpt-3.5-turbo if you don't have GPT-4 access
      messages=[
          {"role": "system", "content": """Your job is to generate system prompts for GPT-4, given a description of the use-case and some test cases.

The prompts you will be generating will be for freeform tasks, such as generating a landing page headline, an intro paragraph, solving a math problem, etc.

In your generated prompt, you should describe how the AI should behave in plain English. Include what it will see, and what it's allowed to output. Be creative with prompts to get the best possible results. The AI knows it's an AI -- you don't need to tell it this.

You will be graded based on the performance of your prompt... but don't cheat! You cannot include specifics about the test cases in your prompt. Any prompts with examples will be disqualified.

Most importantly, output NOTHING but the prompt. Do not include anything else in your message."""},
          {"role": "user", "content": f"Here are some test cases:`{test_cases}`\n\nHere is the description of the use-case: `{description.strip()}`\n\nRespond with your prompt, and nothing else. Be creative."}
          ],
      temperature=.9,
      n=number_of_prompts)

  prompts = []

  for i in outputs.choices:
    prompts.append(i.message.content)
  return prompts

def expected_score(r1, r2):
    return 1 / (1 + 10**((r2 - r1) / 400))

def update_elo(r1, r2, score1):
    e1 = expected_score(r1, r2)
    e2 = expected_score(r2, r1)
    return r1 + K * (score1 - e1), r2 + K * ((1 - score1) - e2)

def test_candidate_prompts(test_cases, description, prompts):
  # Initialize each prompt with an ELO rating of 1200
  prompt_ratings = {prompt: 1200 for prompt in prompts}

  # Calculate total rounds for progress bar
  total_rounds = len(test_cases) * len(prompts) * (len(prompts) - 1) // 2

  # Initialize progress bar
  pbar = tqdm(total=total_rounds, ncols=70)

  # For each pair of prompts
  for prompt1, prompt2 in itertools.combinations(prompts, 2):
      # For each test case
      for test_case in test_cases:
          # Update progress bar
          pbar.update()

          # Generate outputs for each prompt
          generation1 = openai.ChatCompletion.create(
              model='gpt-3.5-turbo',
              messages=[
                  {"role": "system", "content": prompt1},
                  {"role": "user", "content": f"{test_case['prompt']}"}
              ],
              max_tokens=60,
              temperature=0.8,
          ).choices[0].message.content

          generation2 = openai.ChatCompletion.create(
              model='gpt-3.5-turbo',
              messages=[
                  {"role": "system", "content": prompt2},
                  {"role": "user", "content": f"{test_case['prompt']}"}
              ],
              max_tokens=60,
              temperature=0.8,
          ).choices[0].message.content

          # Rank the outputs
          try:
            score1 = openai.ChatCompletion.create(
                model='gpt-3.5-turbo',
                messages=[
                    {"role": "system", "content": """Your job is to rank the quality of two outputs generated by different prompts. The prompts are used to generate a response for a given task.

You will be provided with the task description, the test prompt, and two generations - one for each system prompt.

Rank the generations in order of quality. If Generation A is better, respond with 'A'. If Generation B is better, respond with 'B'.

Remember, to be considered 'better', a generation must not just be good, it must be noticeably superior to the other.

Also, keep in mind that you are a very harsh critic. Only rank a generation as better if it truly impresses you more than the other.

Respond with your ranking, and nothing else. Be fair and unbiased in your judgement."""},
                  {"role": "user", "content": f"""Task: {description.strip()}
Prompt: {test_case['prompt']}
Generation A: {generation1}
Generation B: {generation2}"""}
                ],
                logit_bias={
                      '32': 100,  # 'A' token
                      '33': 100,  # 'B' token
                  },
                max_tokens=1,
                temperature=0,
            ).choices[0].message.content
          except:
            time.sleep(61)
            score1 = openai.ChatCompletion.create(
                model='gpt-3.5-turbo',
                messages=[
                    {"role": "system", "content": """Your job is to rank the quality of two outputs generated by different prompts. The prompts are used to generate a response for a given task.

You will be provided with the task description, the test prompt, and two generations - one for each system prompt.

Rank the generations in order of quality. If Generation A is better, respond with 'A'. If Generation B is better, respond with 'B'.

Remember, to be considered 'better', a generation must not just be good, it must be noticeably superior to the other.

Also, keep in mind that you are a very harsh critic. Only rank a generation as better if it truly impresses you more than the other.

Respond with your ranking, and nothing else. Be fair and unbiased in your judgement."""},
                  {"role": "user", "content": f"""Task: {description.strip()}
Prompt: {test_case['prompt']}
Generation A: {generation1}
Generation B: {generation2}"""}
                ],
                logit_bias={
                      '32': 100,  # 'A' token
                      '33': 100,  # 'B' token
                  },
                max_tokens=1,
                temperature=0,
            ).choices[0].message.content


          try:
            score2 = openai.ChatCompletion.create(
                model='gpt-3.5-turbo',
                messages=[
                    {"role": "system", "content": """Your job is to rank the quality of two outputs generated by different prompts. The prompts are used to generate a response for a given task.

You will be provided with the task description, the test prompt, and two generations - one for each system prompt.

Rank the generations in order of quality. If Generation A is better, respond with 'A'. If Generation B is better, respond with 'B'.

Remember, to be considered 'better', a generation must not just be good, it must be noticeably superior to the other.

Also, keep in mind that you are a very harsh critic. Only rank a generation as better if it truly impresses you more than the other.

Respond with your ranking, and nothing else. Be fair and unbiased in your judgement."""},
                  {"role": "user", "content": f"""Task: {description.strip()}
Prompt: {test_case['prompt']}
Generation A: {generation2}
Generation B: {generation1}"""}
                ],
                logit_bias={
                      '32': 100,  # 'A' token
                      '33': 100,  # 'B' token
                  },
                max_tokens=1,
                temperature=.5,
            ).choices[0].message.content
          except:
            time.sleep(61)
            score2 = openai.ChatCompletion.create(
                model='gpt-3.5-turbo',
                messages=[
                    {"role": "system", "content": """Your job is to rank the quality of two outputs generated by different prompts. The prompts are used to generate a response for a given task.

You will be provided with the task description, the test prompt, and two generations - one for each system prompt.

Rank the generations in order of quality. If Generation A is better, respond with 'A'. If Generation B is better, respond with 'B'.

Remember, to be considered 'better', a generation must not just be good, it must be noticeably superior to the other.

Also, keep in mind that you are a very harsh critic. Only rank a generation as better if it truly impresses you more than the other.

Respond with your ranking, and nothing else. Be fair and unbiased in your judgement."""},
                  {"role": "user", "content": f"""Task: {description.strip()}
Prompt: {test_case['prompt']}
Generation A: {generation2}
Generation B: {generation1}"""}
                ],
                logit_bias={
                      '32': 100,  # 'A' token
                      '33': 100,  # 'B' token
                  },
                max_tokens=1,
                temperature=.5,
            ).choices[0].message.content

          # Convert scores to numeric values
          score1 = 1 if score1 == 'A' else 0 if score1 == 'B' else 0.5
          score2 = 1 if score2 == 'B' else 0 if score2 == 'A' else 0.5

          # Average the scores
          score = (score1 + score2) / 2

          # Update ELO ratings
          r1, r2 = prompt_ratings[prompt1], prompt_ratings[prompt2]
          r1, r2 = update_elo(r1, r2, score)
          prompt_ratings[prompt1], prompt_ratings[prompt2] = r1, r2

          # Print the winner of this round
          if score > 0.5:
              print(f"Winner: {prompt1}")
          elif score < 0.5:
              print(f"Winner: {prompt2}")
          else:
              print("Draw")

  # Close progress bar
  pbar.close()

  return prompt_ratings



def generate_optimal_prompt(description, test_cases, number_of_prompts=10):
  prompts = generate_candidate_prompts(description, test_cases, number_of_prompts)
  prompt_ratings = test_candidate_prompts(test_cases, description, prompts)

  # Print the final ELO ratings
  table = PrettyTable()
  table.field_names = ["Prompt", "Rating"]
  for prompt, rating in sorted(prompt_ratings.items(), key=lambda item: item[1], reverse=True):
      table.add_row([prompt, rating])

  print(table)

# In the cell below, fill in your description, test cases, and number of prompts to generate.

In [ ]:
description = "Given a prompt, generate a landing page headline." # this style of description tends to work well

test_cases = [
    {
        'prompt': 'Promoting an innovative new fitness app, Smartly',
    },
    {
        'prompt': 'Why a vegan diet is beneficial for your health',
    },
    {
        'prompt': 'Introducing a new online course on digital marketing',
    },
    {
        'prompt': 'Launching a new line of eco-friendly clothing',
    },
    {
        'prompt': 'Promoting a new travel blog focusing on budget travel',
    },
    {
        'prompt': 'Advertising a new software for efficient project management',
    },
    {
        'prompt': 'Introducing a new book on mastering Python programming',
    },
    {
        'prompt': 'Promoting a new online platform for learning languages',
    },
    {
        'prompt': 'Advertising a new service for personalized meal plans',
    },
    {
        'prompt': 'Launching a new app for mental health and mindfulness',
    }
]


number_of_prompts = 10 # this determines how many candidate prompts to generate... the higher, the more expensive, but the better the results will be

generate_optimal_prompt(description, test_cases, number_of_prompts)